In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations with Mixup (augmentation) and normalization
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = datasets.ImageFolder(root="/kaggle/input/recycle-with-split/recycle_split/train", transform=transform_train)
val_dataset = datasets.ImageFolder(root="/kaggle/input/recycle-with-split/recycle_split/test", transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [3]:
import torch
import torch.nn as nn
import torchvision.models as models

# Define the number of classes for your classification task
num_classes = 4  # Update this with the actual number of classes

# Load the ResNet model pre-trained on ImageNet
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer to match the number of classes
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Example of defining a label smoothing loss function
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 187MB/s] 


In [5]:
# Training the model
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

# Validation loop
model.eval()
val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        val_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

val_loss /= total
val_accuracy = 100 * correct / total
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')


Epoch [1/30], Loss: 1.1238, Accuracy: 56.47%
Epoch [2/30], Loss: 1.0435, Accuracy: 61.76%
Epoch [3/30], Loss: 0.9946, Accuracy: 64.77%
Epoch [4/30], Loss: 0.9469, Accuracy: 67.64%
Epoch [5/30], Loss: 0.9281, Accuracy: 69.01%
Epoch [6/30], Loss: 0.8960, Accuracy: 70.94%
Epoch [7/30], Loss: 0.8839, Accuracy: 71.19%
Epoch [8/30], Loss: 0.8673, Accuracy: 72.46%
Epoch [9/30], Loss: 0.8648, Accuracy: 72.97%
Epoch [10/30], Loss: 0.8509, Accuracy: 73.62%
Epoch [11/30], Loss: 0.8340, Accuracy: 74.80%
Epoch [12/30], Loss: 0.8216, Accuracy: 75.23%
Epoch [13/30], Loss: 0.8128, Accuracy: 75.89%
Epoch [14/30], Loss: 0.8021, Accuracy: 77.10%
Epoch [15/30], Loss: 0.7956, Accuracy: 76.48%
Epoch [16/30], Loss: 0.7836, Accuracy: 77.58%
Epoch [17/30], Loss: 0.7769, Accuracy: 77.73%
Epoch [18/30], Loss: 0.7737, Accuracy: 77.65%
Epoch [19/30], Loss: 0.7507, Accuracy: 79.46%
Epoch [20/30], Loss: 0.7640, Accuracy: 78.72%
Epoch [21/30], Loss: 0.7540, Accuracy: 79.11%
Epoch [22/30], Loss: 0.7395, Accuracy: 80.3